In [ ]:
import os
import scipy.signal as signal
import numpy as np
import pandas as pd
from sklearn import preprocessing
import pywt
import scipy
import scipy.io
from scipy.ndimage.interpolation import zoom
from scipy.fftpack import fft

from utils import *

In [ ]:
mat_file = r'data/mill/mill.mat'
mat = scipy.io.loadmat(mat_file)
mat = mat['mill'][0]

condition_name = ['case', 'run', 'VB', 'time', 'DOC', 'feed', 'material'] 
signal_name = ['smcAC', 'smcDC', 'vib_table', 'vib_spindle', 'AE_table', 'AE_spindle']

In [ ]:
condition_data = np.array(mat[condition_name].tolist()).squeeze()
condition_dataset = pd.DataFrame(condition_data, columns=condition_name)
signal_dataset = [pd.DataFrame(np.array(data).squeeze().T, columns=signal_name) for data in mat[signal_name].tolist()]

In [ ]:
condition_dataset.info()

In [ ]:
df = signal_dataset[0]
df.info()

In [ ]:
totalscale = 256
wavename = 'morl'
sampling_rate = 250
resize_size = 224

fc = pywt.central_frequency(wavename)
cparam = 2 * fc * totalscale
scales = cparam / np.arange(totalscale, 1, -1)

In [ ]:
signal = np.array(df['smcAC'])
signal = signal[(len(signal) - 1024) // 2:(len(signal) + 1024) // 2]
cwtmatr, frequencies = pywt.cwt(signal, scales, wavename, 1 / sampling_rate)
plot_cwt(sampling_rate, signal, cwtmatr, frequencies)

In [ ]:
signal = np.array(df['smcDC'])
signal = signal[(len(signal) - 1024) // 2:(len(signal) + 1024) // 2]
cwtmatr, frequencies = pywt.cwt(signal, scales, wavename, 1 / sampling_rate)
plot_cwt(sampling_rate, signal, cwtmatr, frequencies)

In [ ]:
signal = np.array(df['vib_table'])
signal = signal[(len(signal) - 1024) // 2:(len(signal) + 1024) // 2]
cwtmatr, frequencies = pywt.cwt(signal, scales, wavename, 1 / sampling_rate)
plot_cwt(sampling_rate, signal, cwtmatr, frequencies)

In [ ]:
signal = np.array(df['vib_spindle'])
signal = signal[(len(signal) - 1024) // 2:(len(signal) + 1024) // 2]
cwtmatr, frequencies = pywt.cwt(signal, scales, wavename, 1 / sampling_rate)
plot_cwt(sampling_rate, signal, cwtmatr, frequencies)

In [ ]:
signal = np.array(df['AE_table'])
signal = signal[(len(signal) - 1024) // 2:(len(signal) + 1024) // 2]
cwtmatr, frequencies = pywt.cwt(signal, scales, wavename, 1 / sampling_rate)
plot_cwt(sampling_rate, signal, cwtmatr, frequencies)

In [ ]:
signal = np.array(df['AE_spindle'])
signal = signal[(len(signal) - 1024) // 2:(len(signal) + 1024) // 2]
cwtmatr, frequencies = pywt.cwt(signal, scales, wavename, 1 / sampling_rate)
plot_cwt(sampling_rate, signal, cwtmatr, frequencies)

# Useless

In [ ]:
totalscale = 256
wavename = 'morl'
sampling_rate = 5000

fc = pywt.central_frequency(wavename)
cparam = 2 * fc * totalscale
scales = cparam / np.arange(totalscale, 0, -1)

In [ ]:
path = r'data/c1'
sample_number = 3
column_number = 6
num_para = 1
length_para = 1024

columns = ['Fx', 'Fy', 'Fz', 'Ax', 'Ay', 'Az', 'AE_rms']

data_train = np.empty([sample_number * num_para, column_number, length_para])

for i in range(sample_number):
    data_path = os.path.join(path, f'c_1_{i + 1:03d}.csv')
    csv_data = pd.read_csv(data_path, names=columns)

    for k in range(column_number):
        scaler = preprocessing.StandardScaler()

        column_data = csv_data.iloc[:, k]
        column_data = np.array(column_data).reshape(-1, 1)

        column_data = scaler.fit_transform(column_data.reshape(-1, 1))
        column_data = column_data.flatten()

        for j in range(num_para):
            # 在第i个文件第k列的数据中，从正中间截取1段长度为length_para的数据  #
            start = (column_data.shape[0] // 2) - (length_para // 2)
            end = (column_data.shape[0] // 2) + (length_para // 2)
            data_train[i * num_para + j:, k, :] = column_data[start: end]

data_train = data_train[:, np.newaxis]

In [ ]:
for data in data_train:
    cwtmatrix = np.empty([data.shape[0], totalscale, data.shape[1], data.shape[2]])
    for i in range(data.shape[2]):
        for j in range(data.shape[1]):
            dataset = data[j, :, i]
            cwtmatr, frequencies = pywt.cwt(dataset, scales, wavename, 1.0 / sampling_rate)
            cwtmatrix[j, :, :, i] = cwtmatr

In [ ]:
sample = data_train[0, 0, 0, :]
sample.shape

In [ ]:
cwtmatr, frequencies = pywt.cwt(sample, scales, wavename, 1.0 / sampling_rate)
cwtmatr.shape

In [ ]:
plot_cwt(np.arange(1024) / sampling_rate, sample, cwtmatr, frequencies)

In [ ]:
# -*- coding: utf-8 -*-
import matplotlib.pyplot as plt
import numpy as np
import pywt
from matplotlib.font_manager import FontProperties


sampling_rate = 1024
t = np.arange(0, 1.0, 1.0 / sampling_rate)
f1 = 100
f2 = 200
f3 = 300
data = np.piecewise(t, [t < 1, t < 0.8, t < 0.3],
                    [lambda t: np.sin(2 * np.pi * f1 * t), lambda t: np.sin(2 * np.pi * f2 * t),
                     lambda t: np.sin(2 * np.pi * f3 * t)])
wavename = 'cgau8'
totalscal = 256
fc = pywt.central_frequency(wavename)
cparam = 2 * fc * totalscal
scales = cparam / np.arange(totalscal, 1, -1)
[cwtmatr, frequencies] = pywt.cwt(data, scales, wavename, 1.0 / sampling_rate)
plt.figure(figsize=(8, 4))
plt.subplot(211)
plt.plot(t, data)
plt.xlabel(u"time(s)")
plt.title(u"300Hz 200Hz 100Hz Time spectrum")
plt.subplot(212)
plt.contourf(t, frequencies, abs(cwtmatr))
plt.ylabel(u"freq(Hz)")
plt.xlabel(u"time(s)")
plt.subplots_adjust(hspace=0.4)
plt.show()

In [ ]:
data.shape